In [ ]:
import os
import logging

if not os.path.exists("Logs"):
    os.makedirs("Logs")

def create_logger(name, filename):
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)

    file_handler = logging.FileHandler("Logs/" + filename)

    formatter = logging.Formatter('%(asctime)s | %(name)s | %(levelname)s | %(message)s')
    file_handler.setFormatter(formatter)

    logger.addHandler(file_handler)
    return logger

sqldb_logger = create_logger("sqldb", "sqldb.log")
ui_logger = create_logger("ui", "ui.log")
frontend_logger = create_logger("frontend", "frontend.log")
backend_logger = create_logger("backend", "backend.log")
flask_logger = create_logger("frontend.flask", "frontend_flask.log")

sqldb_logger.info("Database connected")
sqldb_logger.warning("Slow query")
sqldb_logger.error("Database corruption")
sqldb_logger.critical("Database failure detected")

ui_logger.info("UI started")
ui_logger.warning("Slow response")
ui_logger.error("Render error")
ui_logger.critical("UI crash")

frontend_logger.info("Frontend up")
frontend_logger.warning("Input mismatch")
frontend_logger.error("Frontend error")
frontend_logger.critical("Frontend down")

backend_logger.info("Backend up")
backend_logger.warning("Slow response")
backend_logger.error("Request failed")
backend_logger.critical("Backend down")

flask_logger.info("Flask running")
flask_logger.warning("Session timeout")
flask_logger.error("Submission error")
flask_logger.critical("Flask crash")

print("Logs are in the 'Logs' folder.")

Logs are in the 'Logs' folder.


In [ ]:
import json
import os

def read_log_file(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
        return lines
    except FileNotFoundError:
        print(f"Error: The file {file_path} was not found.")
        return []
    except IOError:
        print("Error: Unable to read the file.")
        return []

def parse_log_line(line):

    parts = line.strip().split(" | ")
    if len(parts) == 4:
        timestamp, log_name, log_level, message = parts
        return {"timestamp": timestamp, "log_name": log_name, "log_level": log_level, "message": message}
    return None

def count_log_levels(parsed_logs):
    log_counts = {}
    for log in parsed_logs:
        level = log["log_level"]
        message = log["message"]

        if level not in log_counts:
            log_counts[level] = {}
        if message not in log_counts[level]:
            log_counts[level][message] = 0

        log_counts[level][message] += 1
    return log_counts

def save_to_json(data, output_file):
    try:
        with open(output_file, 'w') as file:
            json.dump(data, file, indent=4)
        print(f"Data saved to {output_file}")
    except IOError:
        print("Error: Unable to write to file.")

log_file_path = "Logs/sqldb.log" 
output_json = "log_levels.json"   

if not os.path.exists("Logs"):
    os.makedirs("Logs")

log_lines = read_log_file(log_file_path)

parsed_logs = [parse_log_line(line) for line in log_lines if parse_log_line(line)]

log_level_counts = count_log_levels(parsed_logs)

save_to_json(log_level_counts, output_json)

Data saved to log_levels.json


In [11]:
import time
import os
import json
import matplotlib.pyplot as plt

def monitor_log_file(json_file_path):
    last_modified = None
    while True:
        try:
            current_modified = os.path.getmtime(json_file_path)
            if current_modified != last_modified:
                last_modified = current_modified
                with open(json_file_path, 'r') as file:
                    data = json.load(file)
                    critical_messages = data.get('CRITICAL', {})
                    print("\nUpdated Critical Messages:")
                    for message, count in critical_messages.items():
                        print(f"Critical message: {message} - Occurrences: {count}")
        except FileNotFoundError:
            print(f"Error: The file {json_file_path} does not exist.")
        except json.JSONDecodeError:
            print("Error: The JSON file format is invalid.")
        time.sleep(5) 

def plot_log_levels(json_file_path):
    try:
        with open(json_file_path, 'r') as file:
            data = json.load(file)
        
        levels = ['INFO', 'WARNING', 'ERROR', 'CRITICAL']
        counts = [len(data.get(level, {})) for level in levels]
        
        plt.bar(levels, counts, color=['blue', 'yellow', 'red', 'purple'])
        plt.title('Log Levels Distribution')
        plt.xlabel('Log Level')
        plt.ylabel('Number of Messages')
        plt.show()
    except FileNotFoundError:
        print(f"Error: The file {json_file_path} does not exist.")
    except json.JSONDecodeError:
        print("Error: The JSON file format is invalid.")

json_file_path = "log_levels.json"  

monitor_log_file(json_file_path)

plot_log_levels(json_file_path)


Updated Critical Messages:
Critical message: Database failure - Occurrences: 1
Critical message: Database failure detected - Occurrences: 14


KeyboardInterrupt: 